# Pipelines de dados: a chave para modelos de machine learning mais robustos e escaláveis

Neste tutorial, vamos dissecar um script Python que demonstra a construção de um pipeline de Machine Learning para prever preços de casas. O objetivo é entender como lidar com diferentes tipos de features (numéricas e categóricas), tratar valores ausentes e integrar todas essas etapas em um fluxo de trabalho coeso e eficiente usando sklearn.pipeline e sklearn.compose.ColumnTransformer.

**Pré-requisitos**

*   Python 3.x
*   Bibliotecas: pandas, scikit-learn, learntools.core (específico do Kaggle Learn)

## Etapa 1: Configuração Inicial e Carregamento dos Dados
Esta etapa configura o ambiente e carrega os conjuntos de dados de treino e teste.


In [15]:
# Import helpful libraries
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

import pandas as pd

# Load the training data
train_data_path = '/content/sample_data/california_housing_train.csv'
X_full = pd.read_csv(train_data_path)

# Load the test data
test_data_path = '/content/sample_data/california_housing_test.csv'
X_test_full = pd.read_csv(test_data_path)

# Display the first few rows of the training data
print("Training Data Head:")
display(train_data.head())

# Display the first few rows of the test data
print("\nTest Data Head:")
display(test_data.head())

Training Data Head:


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0



Test Data Head:


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.05,37.37,27.0,3885.0,661.0,1537.0,606.0,6.6085,344700.0
1,-118.30,34.26,43.0,1510.0,310.0,809.0,277.0,3.5990,176500.0
2,-117.81,33.78,27.0,3589.0,507.0,1484.0,495.0,5.7934,270500.0
3,-118.36,33.82,28.0,67.0,15.0,49.0,11.0,6.1359,330000.0
4,-119.67,36.33,19.0,1241.0,244.0,850.0,237.0,2.9375,81700.0


## Etapa 2: Preparação e Divisão dos Dados
Nesta etapa, separamos a variável alvo (median_house_value), removemos linhas com valores ausentes no alvo e dividimos os dados em conjuntos de treino e validação.

In [17]:
# --- Etapa 2: Preparação e Divisão dos Dados ---

# Remove linhas onde a coluna 'median_house_value' (nosso alvo) tem valores ausentes.
# 'axis=0' significa remover linhas, 'inplace=True' modifica o DataFrame diretamente.
X_full.dropna(axis=0, subset=['median_house_value'], inplace=True)

# Separa a variável alvo (y) dos preditores (X).
# 'y' contém apenas os preços de venda.
y = X_full.median_house_value

# Remove a coluna 'median_house_value' do DataFrame de features 'X_full'.
# 'axis=1' significa remover colunas, 'inplace=True' modifica o DataFrame.
X_full.drop(['median_house_value'], axis=1, inplace=True)

print(f"\nDimensão de X_full após remover 'median_house_value' e NaN's: {X_full.shape}")


Dimensão de X_full após remover 'median_house_value' e NaN's: (17000, 8)


In [18]:
# Divide os dados completos em conjuntos de treino e validação.
# X_full -> features, y -> alvo
# train_size=0.8: 80% dos dados para treino
# test_size=0.2: 20% dos dados para validação
# random_state=0: Garante que a divisão seja a mesma cada vez que o código é executado.
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y,
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

print(f"Dimensão de X_train_full: {X_train_full.shape}")
print(f"Dimensão de X_valid_full: {X_valid_full.shape}")

Dimensão de X_train_full: (13600, 8)
Dimensão de X_valid_full: (3400, 8)


**Explicação**:

*   **X_full.dropna(axis=0, subset=['median_house_value'], inplace=True)**: Garante que não haja valores NaN na variável alvo, o que inviabilizaria o treinamento do modelo para essas instâncias.

*   **y = X_full.median_house_value**: Cria uma Series pandas para a variável alvo.

*  **X_full.drop(['median_house_value'], axis=1, inplace=True)**: Remove a variável alvo do conjunto de features.

*   **train_test_split**: Divide os dados em conjuntos de treino e validação. O conjunto de treino é usado para ensinar o modelo, e o conjunto de validação é usado para avaliar seu desempenho em dados que ele não "viu" durante o treinamento. random_state garante a reprodutibilidade.

## Etapa 3: Identificação de Tipos de Features
Esta etapa categoriza as colunas em numéricas e categóricas para aplicar o pré-processamento adequado a cada tipo.


In [20]:
# --- Etapa 3: Identificação de Tipos de Features ---

# "Cardinalidade" significa o número de valores únicos em uma coluna.
# Seleciona colunas categóricas com cardinalidade relativamente baixa (menos de 10 valores únicos)
# e que tenham o tipo de dado 'object' (geralmente strings).
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and
                    X_train_full[cname].dtype == "object"]
print(f"\nColunas Categóricas ({len(categorical_cols)}):")
print(categorical_cols)

# Seleciona colunas numéricas (tipo de dado 'int64' ou 'float64').
numerical_cols = [cname for cname in X_train_full.columns if
                X_train_full[cname].dtype in ['int64', 'float64']]
print(f"\nColunas Numéricas ({len(numerical_cols)}):")
print(numerical_cols)

# Combina as listas de colunas selecionadas.
my_cols = categorical_cols + numerical_cols
print(f"\nTotal de Colunas Selecionadas: {len(my_cols)}")


Colunas Categóricas (0):
[]

Colunas Numéricas (8):
['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income']

Total de Colunas Selecionadas: 8


In [21]:
# Mantém apenas as colunas selecionadas nos DataFrames de treino, validação e teste.
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

In [22]:
# Exibe as primeiras 5 linhas do DataFrame de treino com as colunas selecionadas
print("\n--- X_train.head() com colunas selecionadas ---")
print(X_train.head())
print(f"Dimensão de X_train após seleção de colunas: {X_train.shape}")


--- X_train.head() com colunas selecionadas ---
       longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
12036    -121.41     38.57                16.0       4429.0          1124.0   
8210     -118.44     34.19                19.0       3487.0           959.0   
15153    -122.26     37.77                52.0       1565.0           315.0   
8022     -118.42     34.24                36.0       1181.0           220.0   
133      -116.06     34.15                15.0      10377.0          2331.0   

       population  households  median_income  
12036      1538.0       960.0         3.2443  
8210       2278.0       835.0         2.6709  
15153       637.0       297.0         4.7778  
8022        775.0       218.0         4.7228  
133        4507.0      1807.0         2.2466  
Dimensão de X_train após seleção de colunas: (13600, 8)


**Explicação**:



*   **Colunas Categóricas:** São identificadas aquelas que são do tipo object (geralmente strings) e que possuem poucos valores únicos (nunique() < 10). Uma cardinalidade baixa sugere que a codificação One-Hot será eficiente.

*   **Colunas Numéricas:** São identificadas pelo tipo int64 ou float64.

*   **my_cols:** Lista final das features que serão usadas no modelo.

*   **X_train, X_valid, X_test:** Os DataFrames são filtrados para conter apenas as colunas selecionadas. .copy() é usado para evitar avisos de SettingWithCopyWarning em operações futuras.

## Etapa 4: Construção dos Transformadores de Pré-processamento
Aqui definimos como tratar valores ausentes e como converter features categóricas em numéricas.


In [23]:
# --- Etapa 4: Construção dos Transformadores de Pré-processamento ---

# Importa as classes necessárias do Scikit-learn para pré-processamento e pipelines.
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [24]:
# 1. Pré-processamento para dados numéricos
# Usamos SimpleImputer para preencher valores ausentes.
# strategy='constant' preencherá com um valor constante (o padrão é 0 ou NaN se fill_value não for especificado).
# Neste caso, para features numéricas, 'constant' preencheria com 0 por padrão.
numerical_transformer = SimpleImputer(strategy='constant')
print("\n--- numerical_transformer ---")
print(numerical_transformer)


--- numerical_transformer ---
SimpleImputer(strategy='constant')


In [25]:
# 2. Pré-processamento para dados categóricos
# Criamos um Pipeline para aplicar múltiplos passos sequencialmente.
categorical_transformer = Pipeline(steps=[
    # Primeiro passo: Imputação de valores ausentes.
    # strategy='most_frequent' preenche valores ausentes com o valor mais comum da coluna.
    ('imputer', SimpleImputer(strategy='most_frequent')),
    # Segundo passo: Codificação One-Hot.
    # Converte variáveis categóricas em um formato numérico que os modelos podem entender.
    # handle_unknown='ignore': Se um valor categórico novo for encontrado nos dados de teste
    # que não foi visto nos dados de treino, ele será ignorado (as colunas correspondentes terão valor 0).
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
print("\n--- categorical_transformer ---")
print(categorical_transformer)


--- categorical_transformer ---
Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('onehot', OneHotEncoder(handle_unknown='ignore'))])


**Explicação**:

*   **SimpleImputer:** Ferramenta para preencher (imputar) valores ausentes.
*  *	Para numéricos: strategy='constant' preenche com um valor fixo (0 é o padrão para constant se não especificado).
*  *  Para categóricos: strategy='most_frequent' preenche com a categoria mais comum.

*   **OneHotEncoder:** Converte features categóricas em representações numéricas binárias. Por exemplo, se uma coluna 'Cor' tem valores 'Vermelho', 'Azul', 'Verde', One-Hot Encoder criará três novas colunas ('Cor_Vermelho', 'Cor_Azul', 'Cor_Verde') com 0s e 1s. handle_unknown='ignore' é crucial para lidar com categorias que podem aparecer nos dados de teste/validação, mas não foram vistas nos dados de treino.

*   Pipeline (para categóricos): É usado aqui para agrupar as etapas de imputação e codificação One-Hot para as features categóricas, garantindo que elas sejam aplicadas na ordem correta.


## Etapa 5: Agrupamento dos Transformadores com ColumnTransformer
Esta é uma parte chave: o ColumnTransformer permite aplicar diferentes transformadores a diferentes subconjuntos de colunas simultaneamente.


In [26]:
# --- Etapa 5: Agrupamento dos Transformadores com ColumnTransformer ---

# Agrupa o pré-processamento para dados numéricos e categóricos.
# ColumnTransformer permite aplicar diferentes transformadores a diferentes conjuntos de colunas.
preprocessor = ColumnTransformer(
    transformers=[
        # Aplica 'numerical_transformer' às 'numerical_cols'
        ('num', numerical_transformer, numerical_cols),
        # Aplica 'categorical_transformer' às 'categorical_cols'
        ('cat', categorical_transformer, categorical_cols)
    ])
print("\n--- preprocessor (ColumnTransformer) ---")
print(preprocessor)



--- preprocessor (ColumnTransformer) ---
ColumnTransformer(transformers=[('num', SimpleImputer(strategy='constant'),
                                 ['longitude', 'latitude', 'housing_median_age',
                                  'total_rooms', 'total_bedrooms', 'population',
                                  'households', 'median_income']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 [])])


**Explicação**:

*   ColumnTransformer: Esta classe é essencial para lidar com pipelines complexos onde diferentes tipos de colunas exigem diferentes estratégias de pré-processamento.

*   transformers: É uma lista de tuplas, onde cada tupla contém:

*  * Um nome ('num', 'cat').
*  * Um objeto transformador (numerical_transformer, categorical_transformer).
*  * Uma lista de nomes de colunas às quais o transformador deve ser aplicado (numerical_cols, categorical_cols).

## Etapa 6: Definição do Modelo de Machine Learning
Agora definimos o modelo que será usado para fazer as previsões.


In [27]:
# --- Etapa 6: Definição do Modelo de Machine Learning ---

# Define o modelo de Regressão de Floresta Aleatória.
# n_estimators=100: Usa 100 árvores de decisão na floresta.
# random_state=0: Garante a reprodutibilidade dos resultados.
model = RandomForestRegressor(n_estimators=100, random_state=0)
print("\n--- Modelo (RandomForestRegressor) ---")
print(model)


--- Modelo (RandomForestRegressor) ---
RandomForestRegressor(random_state=0)


**Explicação**:

*   **RandomForestRegressor**: Um algoritmo de Machine Learning de ensemble que combina várias árvores de decisão para fazer previsões. É robusto e geralmente tem bom desempenho.

*   **n_estimators**: Número de árvores na floresta. Mais árvores geralmente levam a um desempenho melhor, mas aumentam o tempo de computação.

*   **random_state**: Define a semente para a geração de números aleatórios, garantindo que o modelo produza os mesmos resultados a cada execução.


## Etapa 7: Construção do Pipeline Completo (Pré-processamento + Modelo)
Esta é a parte onde tudo se une: o Pipeline principal que agrupa o pré-processador e o modelo.


In [28]:
# --- Etapa 7: Construção do Pipeline Completo ---

# Agrupa o pré-processamento e o código de modelagem em um único Pipeline.
# Isso garante que as etapas de pré-processamento (imputação, one-hot encoding)
# sejam aplicadas consistentemente tanto aos dados de treino quanto aos de validação/teste.
clf = Pipeline(steps=[('preprocessor', preprocessor), # Primeiro o pré-processador
                      ('model', model)                # Depois o modelo
                     ])
print("\n--- Pipeline Completo (clf) ---")
print(clf)


--- Pipeline Completo (clf) ---
Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='constant'),
                                                  ['longitude', 'latitude',
                                                   'housing_median_age',
                                                   'total_rooms',
                                                   'total_bedrooms',
                                                   'population', 'households',
                                                   'median_income']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                     

**Explicação**:

*	**Pipeline**: O coração da organização do fluxo de trabalho de ML. Ele encadeia várias etapas em uma única unidade.
*	**steps**: Uma lista de tuplas, onde cada tupla contém um nome (string) e um objeto transformador ou modelo. As etapas são executadas sequencialmente. O benefício principal é evitar o vazamento de dados (data leakage) e tornar o código mais limpo e modular.


# Etapa 8: Treinamento do Modelo
Agora treinamos o pipeline completo nos dados de treino.


In [29]:
# --- Etapa 8: Treinamento do Modelo ---

# Pré-processa os dados de treino e ajusta o modelo.
# Quando 'fit' é chamado no pipeline, ele:
# 1. Chama 'fit_transform' no 'preprocessor' usando X_train.
#    Isso aprende os parâmetros de imputação (ex: moda, constante) e os mapeamentos do OneHotEncoder.
#    Aplica as transformações para gerar X_train_processed.
# 2. Em seguida, chama 'fit' no 'model' usando X_train_processed e y_train.
print("\n--- Treinando o Pipeline... ---")
clf.fit(X_train, y_train)
print("Treinamento Completo!")



--- Treinando o Pipeline... ---
Treinamento Completo!


**Explicação**:

* **clf.fit(X_train, y_train)**: Este é o comando mágico. Quando você chama fit no pipeline, ele internamente chama fit_transform em todas as etapas de pré-processamento (apenas fit na última etapa, que é o modelo). Isso garante que o pré-processamento seja aprendido apenas com os dados de treino, evitando vazamento de informações do conjunto de validação ou teste.


## Etapa 9: Realização de Previsões e Avaliação
Após o treinamento, usamos o pipeline para fazer previsões nos dados de validação e avaliamos seu desempenho.


In [30]:
# --- Etapa 9: Realização de Previsões e Avaliação ---

# Pré-processa os dados de validação e obtém previsões.
# Quando 'predict' é chamado no pipeline, ele:
# 1. Chama 'transform' no 'preprocessor' usando X_valid.
#    Ele aplica as transformações aprendidas no passo 'fit' (NÃO APRENDE NOVAS COISAS).
#    Gera X_valid_processed.
# 2. Em seguida, chama 'predict' no 'model' usando X_valid_processed.
preds = clf.predict(X_valid)

# Exibe os últimos 5 valores reais de median_house_value no conjunto completo (apenas para referência)
print("\nÚltimos 5 valores reais de 'median_house_value' (y.tail()):")
print(y.tail())

# Exibe os últimos 5 valores previstos no conjunto de validação
print("\nÚltimos 5 valores previstos (preds[-5:]):")
print(preds[-5:])


Últimos 5 valores reais de 'median_house_value' (y.tail()):
16995    111400.0
16996     79000.0
16997    103600.0
16998     85800.0
16999     94600.0
Name: median_house_value, dtype: float64

Últimos 5 valores previstos (preds[-5:]):
[219178.   239364.01 136810.   105306.    68507.  ]


In [31]:
# Calcula o Erro Absoluto Médio (MAE) entre os valores reais e previstos.
# O MAE é uma métrica que mede a média das diferenças absolutas entre as previsões e os valores reais.
mae = mean_absolute_error(y_valid, preds)
print(f'\nMAE: {mae}')



MAE: 31643.480294117646


**Explicação**:

* **clf.predict(X_valid)**: Faz previsões. Similar ao fit, ele aplica o transform nas etapas de pré-processamento (usando os parâmetros aprendidos no treino) e, em seguida, o predict no modelo.

*	**mean_absolute_error**: Uma métrica comum para problemas de regressão. Ela mede a média das magnitudes dos erros em um conjunto de previsões, sem considerar a direção dos erros. Um MAE menor indica um modelo melhor.


## Etapa 10: Melhorando o Desempenho (Sua Vez!)
O trecho do Kaggle Learning System pede para você tentar melhorar o MAE. Isso pode ser feito ajustando os parâmetros dos transformadores ou do modelo.


In [32]:
# --- Etapa 10: Melhorando o Desempenho ---

# Você pode experimentar diferentes estratégias de imputação ou parâmetros do modelo.

# Exemplo de possíveis melhorias:

# 1. Mudar a estratégia de imputação para dados numéricos:
# numerical_transformer = SimpleImputer(strategy='mean') # Experimente 'mean' ou 'median'

# 2. Ajustar os parâmetros do modelo RandomForestRegressor:
# model = RandomForestRegressor(n_estimators=200, random_state=0, max_depth=7) # Mais estimadores, profundidade máxima

# Para este tutorial, vamos manter as configurações originais conforme o exemplo inicial,
# mas esta é a seção onde você faria suas otimizações.
numerical_transformer = SimpleImputer(strategy='constant') # Mantido como no exemplo
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
]) # Mantido como no exemplo

model = RandomForestRegressor(n_estimators=100, random_state=0) # Mantido como no exemplo

# Reconstruir o preprocessor e o pipeline com as novas configurações (se houver)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

print("\n--- Re-treinando o Pipeline (com possíveis ajustes) ---")
my_pipeline.fit(X_train, y_train)

preds = my_pipeline.predict(X_valid)
score = mean_absolute_error(y_valid, preds)
print(f'MAE após possíveis ajustes (ou sem ajustes se mantidas as originais): {score}')



--- Re-treinando o Pipeline (com possíveis ajustes) ---
MAE após possíveis ajustes (ou sem ajustes se mantidas as originais): 31643.480294117646


**Explicação**:

*	Esta etapa é um convite à experimentação. Você pode tentar diferentes strategy para SimpleImputer (e.g., 'mean', 'median').
*	Para o RandomForestRegressor, pode-se ajustar n_estimators (número de árvores), max_depth (profundidade máxima das árvores), min_samples_split (número mínimo de amostras necessárias para dividir um nó interno), entre outros. O objetivo é encontrar uma combinação que minimize o MAE no conjunto de validação.


## Etapa 11: Gerando Previsões para Dados de Teste
Finalmente, usamos o pipeline treinado para gerar previsões no conjunto de dados de teste, que não tem a variável alvo (median_house_value).


In [33]:
# --- Etapa 11: Gerando Previsões para Dados de Teste ---

# O pipeline 'my_pipeline' já está treinado do passo anterior.
# Agora, aplicamos ele aos dados de teste (X_test).
# Note que X_test_full foi filtrado para 'my_cols' no passo 3, resultando em X_test.
print("\n--- Gerando previsões para os dados de teste ---")
preds_test =  my_pipeline.predict(X_test)

print("\nÚltimos 5 valores previstos nos dados de teste:")
print(preds_test[-5:])


--- Gerando previsões para os dados de teste ---

Últimos 5 valores previstos nos dados de teste:
[278272.   231517.    71402.   238549.14 498014.94]


**Explicação**:

*	**my_pipeline.predict(X_test)**: Aplica o pipeline treinado (pré-processamento + modelo) aos dados de teste. É crucial que o pré-processamento para os dados de teste seja exatamente o mesmo (usando os mesmos parâmetros aprendidos) que foi aplicado aos dados de treino e validação.

*	**preds_test**: Contém as previsões dos preços de casas para o conjunto de teste.


## Conclusão

**Parabéns**! Você passou por um exemplo completo de construção de um pipeline de Machine Learning para um problema de dados tabulares. Entendeu como:
1.	**Carregar e Preparar Dados**: Separar features e alvo, dividir em treino/validação.
2.	**Identificar Tipos de Features**: Distinguir colunas numéricas de categóricas.
3.	**Construir Transformadores**: Usar SimpleImputer para dados ausentes e OneHotEncoder para categóricos.
4.	**Agrupar Transformadores**: Utilizar ColumnTransformer para aplicar diferentes lógicas a diferentes conjuntos de colunas.
5.	**Definir um Modelo**: Escolher um algoritmo de ML (neste caso, RandomForestRegressor).
6.	**Criar um Pipeline Completo**: Unir pré-processamento e modelo em uma sequência lógica com Pipeline.
7.	**Treinar, Prever e Avaliar**: Usar o pipeline para treinar o modelo, fazer previsões e medir seu desempenho.
8.	**Gerar Previsões de Teste**: Aplicar o pipeline treinado a dados não vistos para prever resultados finais.
Este padrão de pipeline é extremamente poderoso e recomendado para qualquer projeto de Machine Learning, pois garante consistência, evita vazamento de dados e torna o código mais legível e mantenível.

Agora você pode experimentar diferentes transformadores, estratégias de imputação, codificadores, modelos e seus hiperparâmetros para buscar um desempenho ainda melhor!

